In [8]:
import os
import openai
import pandas as pd
from dotenv import dotenv_values

config = dotenv_values(".env")

openai.organization = config["ORG_ID"]
openai.api_key = config["OPENAI_API_KEY"]

schema_file = "/Users/rohitmishra/Desktop/lom_athena_schema.json"

df = pd.read_json(schema_file)

query = "How many offers do not have any applications in the past 2 days"

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct AWS Athena SQL query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Use the following data dump for the table schema:

{table_info}

Question: {input}"""

_dict = {"input": query, "table_info": df.to_string}
_prompt =  _DEFAULT_TEMPLATE.format(**_dict)

print(_prompt)

def generate_sql_query(prompt, max_tokens=100, temperature=0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=["\n"]):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty
    )
    print(response)
    return response

athena_query = generate_sql_query(_prompt).choices[0].text.strip()

print(athena_query)


Given an input question, first create a syntactically correct AWS Athena SQL query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Use the following data dump for the table schema:

<bound method DataFrame.to_string of                  table_schema         table_name           column_name  \
0    loan_origination_manager  loan_applications                    id   
1    loan_origination_manager  loan_applications          reference_id   
2    loan_origination_manager  loan_applications     user_reference_id   
3    loan_origination_manager  loan_applications  product_reference_id   
4    loan_origination_manager  loan_applications   applicant_detail_id   
..                        ...                ...                   ...   
425  loan_origination_manager           statuses            created_at   
426  loan_origination